In [1]:
# THERE ARE SEPARATE NOTEBOOKS FOR VISUALIZATIONS, DATASET ANALYSIS, ETC. IN THE REPO.

import pandas as pd
import numpy as np

# READ THE CSV INTO DATAFRAME

df = pd.read_csv('Syngenta/Syngenta_2017/Experiment_dataset.csv')

In [2]:
# CURRENTLY NECESSARY IF: USING 174 ADDITIONAL VARIETY COLUMNS METHOD

# THIS IS A DIFFERENT APPROACH TO THE ABOVE FOUR CELLS, WHERE WE HAVE 174 ADDITIONAL FEATURE COLUMNS
# EACH WITH A 0 (IF IT IS NOT OF THAT VARIETY) OR A 1 (IF IT IS OF THAT VARIETY)

variety_dummies = pd.get_dummies(df.Variety)
df = pd.concat([df, variety_dummies], axis=1)

In [3]:
# GOAL OF THIS MODULE:
# Encode the planting date as a season
# NEW GOAL:
# GET DUMMIES FOR SEASONS

# remove the dates that are "."
df = df[~df['Planting date'].str.match("\.")]
plant_date = df['Planting date'].apply(lambda dt: pd.to_datetime(dt))
plant_months = plant_date.apply(lambda dt: dt.month)
season = plant_date.rename("Season")
season = pd.to_datetime(season)
season = season.apply(lambda dt: (dt.month%12 + 3)//3)
# df['Plant date'] = pd.to_datetime(df['Plant date'])
df = pd.concat([df, season], axis=1)

# plant_date = pd.to_datetime(df['Planting date'], infer_datetime_format=True)
# df = df['Planting date'].apply(lambda dt: (dt.month%12 + 3)//3)
# pd.get_dummies(df['Planting date'])


In [4]:
# ADD MONTH OF MAY AND JUNE ONE HOT ENCODING INTO THE DATAFRAME
pd.get_dummies(plant_months).sum()
june = pd.get_dummies(plant_months).loc[:,6]
june = june.rename("June")
may = pd.get_dummies(plant_months).loc[:,5]
may = may.rename("May")
df = pd.concat([df, may], axis=1)
df = pd.concat([df, june], axis=1)

In [5]:
# LATITUDE AND LONGITUDE CLUSTERING INTO FEATURES

from sklearn.cluster import KMeans

latlong = df.loc[:, ['Latitude', 'Longitude']]

kmeans = KMeans(n_clusters=4, random_state=0).fit(latlong)
kmeans.labels_.shape
lat_long_dummies = pd.get_dummies(kmeans.labels_)
lat_long_dummies = lat_long_dummies.rename(index=int, columns={0: "Loc Clust 0",
                                                               1: "Loc Clust 1",
                                                               2: "Loc Clust 2",
                                                               3: "Loc Clust 3"})
df = pd.concat([df, lat_long_dummies], axis = 1)

In [12]:
#REMOVE ANY NAN VALUES

print(df.columns)
df = df[~df.Silt.isnull()]
df = df[~df['Loc Clust 1'].isnull()]

Index(['Yield', 'Year', 'Temperature', 'Precipitation', 'Solar Radiation',
       'Soil class', 'CEC', 'Organic matter', 'pH', 'Clay',
       ...
       'V156797', 'V156806', 'V156807', 'May', 'June', 'Loc Clust 0',
       'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3', 'YieldBucket'],
      dtype='object', length=194)


In [ ]:
#GOAL OF THIS MODULE:
#TO LIMIT THE NUMBER OF VARIETY COLUMNS NEEDED AS FEATURES, USE BINARY REPRESENTATION

from sklearn import preprocessing
lb = preprocessing.LabelBinarizer()
binarized = lb.fit(df.Variety)
print(binarized)
df.Variety = pd.Series(binarized.transform(df.Variety))
print(binarized.transform(df.Variety).reshape(1,-1))

In [ ]:
for col in df.columns:
    print(col, type(df[col][0]))

In [13]:
# DROP ALL THE CELLS THAT ARE NOT USABLE SUCH AS THE ONES THAT ARE STRINGS OR DATES

# set if want to drop some columns specifically
should_drop = 1
columns_to_drop = ['Experiment', 'Location',
                   'Check Yield', 'Yield difference', 'Latitude',
                   'Longitude', 'Variety', 'PI', 'Planting date', 'Season']

# set if want to keep some columns specifically
should_keep = 0
# columns_to_keep = ['Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3']
columns_to_keep_top = ['Silt', 'Precipitation', 'Temperature', 'Solar Radiation', 'Organic matter']
columns_VARIETIES_ONLY = np.asarray(df.iloc[:, df.columns.str.match('V\d\d\d\d\d\d')].columns)

#set the below variable to whatever columns you want to keep
columns_to_keep = columns_to_keep_top

MUST_HAVE_COLUMNS = ['Yield']
# print(columns_to_keep)

df = df.drop(columns_to_drop, axis=1) if should_drop else df
df = df.loc[:, np.concatenate((columns_to_keep, MUST_HAVE_COLUMNS))] if should_keep else df
df['YieldBucket'] = pd.Series(pd.qcut(df.Yield, q=3, labels=["high", "medium", "low"]))
print("The final dataframe has columns: ", df.columns)

ValueError: labels ['Experiment' 'Location' 'Check Yield' 'Yield difference' 'Latitude'
 'Longitude' 'Variety' 'PI' 'Planting date' 'Season'] not contained in axis

In [11]:
# LET US ALSO MAKE SURE THERE ARE NO NAN IN THE DATA
print("We expect to be %s nan values and there actually are %s nan values\n" % (0, np.sum(df.isnull().sum())))
print(df.isnull().sum())
# AFTER COLUMNS, MAKE SURE NO SKETCHY ONES
for col in df.columns:
    print(col, type(df[col][0]))    

We expect to be 0 nan values and there actually are 378300 nan values

Yield              1950
Year               1950
Temperature        1950
Precipitation      1950
Solar Radiation    1950
Soil class         1950
CEC                1950
Organic matter     1950
pH                 1950
Clay               1950
Silt               1950
Sand               1950
Area               1950
V000016            1950
V000017            1950
V000018            1950
V000023            1950
V000024            1950
V000025            1950
V000030            1950
V000032            1950
V000034            1950
V000036            1950
V000039            1950
V000047            1950
V000050            1950
V000051            1950
V000058            1950
V000060            1950
V000062            1950
                   ... 
V152779            1950
V155180            1950
V155820            1950
V155842            1950
V155843            1950
V155918            1950
V156247            1950
V156305          

In [14]:
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT
# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT# TRAIN AND TEST SPLIT

X = df.drop(['Yield', 'YieldBucket'], axis=1)

print(X.columns)

y = df.Yield

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05, train_size = 0.1, random_state = 42)

INPUT_COLS = X_train.columns
# TEST_COLS = y_train.columns

Index(['Year', 'Temperature', 'Precipitation', 'Solar Radiation', 'Soil class',
       'CEC', 'Organic matter', 'pH', 'Clay', 'Silt',
       ...
       'V156786', 'V156797', 'V156806', 'V156807', 'May', 'June',
       'Loc Clust 0', 'Loc Clust 1', 'Loc Clust 2', 'Loc Clust 3'],
      dtype='object', length=192)


In [15]:
print("X_train shape:", X_train.shape, "\ny_train shape:", y_train.shape)

X_train shape: (7813, 192) 
y_train shape: (7813,)


In [16]:
# This function will evaluate the errors based on RMSE (from the challenge spec)
# also will evaluate based on average error

from sklearn.metrics import mean_squared_error
def evaluate_errors(prediction, actual):
    print("RMSE Error: ", np.sqrt(mean_squared_error(prediction, actual)))
    avg_error_vector = np.absolute(((preds - y_test) / y_test) * 100)
#     print("Average Error: ", np.mean(avg_error_vector))
    print("Average Error details:\n", avg_error_vector.describe())
    return avg_error_vector

In [17]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(n_estimators=20, max_depth=13, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

evaluate_errors(preds, y_test)

RMSE Error:  7.230648964933217
Average Error details:
 count    3907.000000
mean       10.273712
std        10.902952
min         0.005787
25%         3.504977
50%         7.463465
75%        13.390107
max       141.373792
Name: Yield, dtype: float64


[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    0.0s finished


37747     3.318402
34960     6.745501
77631     0.285277
29678     2.371641
58370     6.285895
44709     7.483849
37353     2.087536
11083    10.865165
65678     9.758797
68237     6.487128
69975    12.843164
53136     7.409571
25669    29.816400
22055     6.815375
53416     1.862979
1165     20.048285
21660    25.220308
10943    19.820369
29486     3.817819
73128     3.841617
77434    13.139076
74450     6.527960
29534     9.277164
79136     9.895220
51908     4.277561
6382      8.507752
4497      8.274020
29180     6.889641
40968    13.152110
56236     5.154415
           ...    
55268     9.818288
36424     8.397165
38873     1.053541
53108     0.445981
57176    10.455698
9969      5.625612
63588     3.505989
53502    11.167884
45568     4.174549
5098     13.638104
27685    20.754869
19609    60.007392
31942    13.979377
3338     17.543903
62444     9.761453
67291     6.708281
47450     7.662806
11728     4.445628
3337     20.585995
49107     8.438182
19969    20.259952
55298    12.

In [18]:
# GET OUTPUT OF FEATURE IMPORTANCE
def get_feature_importances(regr):
    feature_importances = regr.feature_importances_
    feature_importances = pd.Series(feature_importances)
    feature_importance_df = pd.DataFrame({'feature': X_train.columns,'feature_importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by=['feature_importance'])
    for index, row in feature_importance_df.iterrows():
        print(row['feature'], 'has importance: ', row['feature_importance'])
get_feature_importances(regr)

V121140 has importance:  0.0
V121097 has importance:  2.8596859520076963e-06
V000036 has importance:  6.0793353871786106e-06
V151273 has importance:  6.285801449910723e-06
V152779 has importance:  1.2189654511586188e-05
V152312 has importance:  1.596049707717542e-05
V121524 has importance:  1.7510543902265844e-05
V152102 has importance:  2.3973597899738836e-05
V139094 has importance:  2.3983845058519083e-05
V113424 has importance:  2.457841899194059e-05
V130305 has importance:  3.293727499560608e-05
V000147 has importance:  3.572359519830296e-05
V120999 has importance:  3.644191642371407e-05
V000080 has importance:  3.991145866587386e-05
V156763 has importance:  4.1178649633552265e-05
V156367 has importance:  4.369949671632177e-05
V120912 has importance:  5.087373334012611e-05
V114951 has importance:  5.2992524010322516e-05
V151337 has importance:  5.601140168717358e-05
V103466 has importance:  5.645016604406419e-05
V131675 has importance:  5.977419733786208e-05
V152734 has importance:

In [ ]:
# THIS WILL ONLY WORK WITH THE BUCKET METHOD

from sklearn.ensemble import RandomForestClassifier
regr = RandomForestClassifier(n_estimators=10, max_depth=20, random_state=0, verbose=1)
regr.fit(X_train, y_train)
preds = regr.predict(X_test)

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, preds))

In [ ]:
import numpy as np
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor

from sklearn.feature_selection import RFECV

classifiers = [
    svm.SVR(),
    MLPRegressor(solver='lbfgs', alpha=1e-5,
                     hidden_layer_sizes=(5, 2), random_state=1),
    linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
    linear_model.LassoLars(),
#     linear_model.ARDRegression(),
#     linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

# estimator = svm.SVR(kernel="linear")

# selector = RFECV(estimator, step=1, cv=5, verbose=1)
# selector = selector.fit(X_train, y_train)
# selector.support_ 
# # array([ True,  True,  True,  True,  True,
# #         False, False, False, False, False], dtype=bool)
# selector.ranking_
# # array([1, 1, 1, 1, 1, 6, 4, 3, 2, 5])


#     print(np.sum(preds - y_test))
#     print(clf.predict(X_test),'\n')
#     print(y_test)
#     print('accuracy score:', accuracy_score(y_test, clf.predict(X_test)), '\n')

In [ ]:
for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)
    errors = evaluate_errors(preds, y_test)
    try:
        get_feature_importances(clf)
    except:
        print("NO FEATURE IMPORTANCE METRIC")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]
from sklearn.metrics import accuracy_score
for item in classifiers:
    print(item)
    clf = item
    clf.fit(scale(X_train), y_train)
    preds = clf.predict(scale(X_test))
    print(accuracy_score(y_test, preds))
#     errors = np.absolute(((preds - y_test) / y_test) * 100)
#     print(errors)
#     print(np.mean(errors))

In [ ]:
import random

NUM_VARIETIES = 174

def best_yield_variety(regr, test_set, random_sel = True, n_samples = 174, print_variety_preds = True):
    
    #create empty df
    dup_df = pd.DataFrame()
    
    #choose a rand sample of input test_set (for dev purposes, wouldn't be used in app)
    test_set_sample = test_set.sample(n= n_samples) if random_sel else test_set
    
    #progress of intensive, long for loop upcoming
    counter = 0
    
    #for loop will, for each row in test_set_sample, duplicate that row by NUM_VARIETIES
    for index, row in test_set_sample.iterrows():
        counter+=1
        print(counter)
        dup_df = dup_df.append([row] * NUM_VARIETIES, ignore_index = True)
        
    #extract the varieties columns
    duplicated_df_varieties = dup_df.loc[:, dup_df.columns.str.match('V\d\d\d\d\d\d')]
    #extract the names of the varieties
    varieties_array = duplicated_df_varieties.columns
    num_expanded_data_pts = duplicated_df_varieties.shape[0]
    #we must have a zeroed matrix of the same shape as the duplicated_df_varieties
    #so that we can input a 1 just once for each variety per 174 rows
    d = np.zeros((duplicated_df_varieties.shape[0], NUM_VARIETIES))
    #make d our dataframe, with columns equal to the varieties
    duplicated_df_varieties = pd.DataFrame(d, columns=varieties_array)
    #for loop will place a 1 just once for each variety per 174 rows (one hot rep)
    for i in range(duplicated_df_varieties.shape[0]):
        var_index = i % 174
        duplicated_df_varieties.loc[i, varieties_array[var_index]] = 1
    #remove the varieties (will be added back with the new values)
    dup_df = dup_df.drop(varieties_array, axis = 1)
    #add the new values (one hot representations) from above for loop
    dup_df = pd.concat([dup_df, duplicated_df_varieties], axis=1)
    #do prediction on the entire dataframe
    preds_per_variety = regr.predict(dup_df)
    #*******make it into a dataframe where each row will give the performance of each variety
    #with the same environmental conditions*******
    preds_df = pd.DataFrame(preds_per_variety.reshape((int(num_expanded_data_pts/NUM_VARIETIES), NUM_VARIETIES)),
                            columns=varieties_array)
    
    #environmental conditions (everything except the variety data)
    envcond = test_set_sample.drop(varieties_array, axis=1)
    
    #a simple print out for best variety given the environmental conditions
    hr_preds = []
    if print_variety_preds:
        envcond_cols = envcond.columns
        counter = -1
        for idx, row in envcond.iterrows():
            counter+=1
            out = "For environmental conditions:\n%s\nthe best variety is:%s" % (row, preds_df.idxmax(axis=1)[counter])
            hr_preds.append(out)
            print(out)
    
    return preds_df, envcond, hr_preds
    
        
preds_df, envcond, hr_preds = best_yield_variety(regr, X_test, n_samples = 174)

In [ ]:
#find number of times a variety is the maximum in the above prediction dataframe
maxes = preds_df.idxmax(axis=1) # get first max variety per env
# print(pd.get_dummies(maxes).describe())
print(pd.get_dummies(maxes).sum().sort_values(ascending=False))

In [ ]:
print(hr_preds[0])

In [ ]:
preds_df.describe().sort_values(by="mean", axis=1)